<a href="https://colab.research.google.com/github/woojin-blaze/E-commerce-llm/blob/main/blaze_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64

In [ ]:
!pip install --upgrade datasets

In [ ]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"
#base_model = "beomi/OPEN-SOLAR-KO-10.7B"

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/MyDrive/창통설_Q조_프로젝트"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
data			  data_labeling.ipynb  feature_addition.ipynb		generated_text
data_generation.ipynb	  demo		       finetuned_data_evaluation.ipynb	init.ipynb
data_labeling_base.ipynb  eval		       finetuning.ipynb			results


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

저장한 모델 불러와보기

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
my_model_path = '/content/drive/MyDrive/창통설_Q조_프로젝트/results/llama-base-format-3epoch'

my_model = AutoModelForCausalLM.from_pretrained(my_model_path)
my_tokenizer = AutoTokenizer.from_pretrained(my_model_path)

my_model_path2 = '/content/drive/MyDrive/창통설_Q조_프로젝트/results/solar-ko-3epoch'
my_model2 = AutoModelForCausalLM.from_pretrained(my_model_path2)
my_tokenizer2 = AutoTokenizer.from_pretrained(my_model_path2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
 import re

def extract_response(response):

    patterns = [
        (r"### Product Title:\s*(.*?)(?=\n###|$)", "Product Title"),
        (r"### Related Keywords:\s*(.*?)(?=\n###|$)", "Related Keywords"),
        (r"### Detailed Description:\s*(.*?)(?=\n###|$)", "Detailed Description"),
        (r"Product title in Korean:\s*(.*?)(?=\n###|$)", "Product Title"),
        (r"Product Title:\s*(.*?)(?=\n###|$)", "Product Title"),
        (r"Detailed Description\s*:\s*(.*?)(?=\n###|$)", "Detailed Description"),
        (r"Related keywords:\s*(.*?)(?=\n###|$)", "Related Keywords"),
        (r"\[INST\].*?\[/INST\]\s*(.*?)(?=\n###|$)", "General"),
        (r"<s>\[INST\].*?\[/INST\]\s*(.*?)(?=\n###|$)", "General")
    ]

    for pattern, type_ in patterns:
        match = re.search(pattern, response, re.DOTALL)
        if match:
            title = match.group(1).strip()

            if type_ == "Related Keywords":

                title = re.sub(r"[^\[\],가-힣\s\-\d]", "", title).strip()
            else:

                title = re.sub(r"<\/?s>|<\/?INST>|raw source:.*", "", title).strip()
                title = re.sub(r"[^가-힣\s\-\d,]", "", title).strip()

            if re.search(r"[가-힣]", title):
                return title

    return "No text found."



In [ ]:
data1 = "儿童智能手表电话定位防水触屏学生多功能运动计步器GPS追踪蓝色"
data2 = "黑色牛津布商务电脑包男士2023新款多功能大容量简约手提斜挎公文包"
data3 = "猫眼石项链女时尚2024新款简约个性银色链条配饰女士短款锁骨链"

본래는 구글 번역 API를 활용함.


In [ ]:
data1_ko = "어린이 스마트 시계 전화 위치 방수 터치 스크린 학생 다기능 스포츠 보수계 GPS 추적 블루"
data2_ko = "남성용 블랙 옥스포드 천 비즈니스 컴퓨터 가방 2023 새로운 다기능 대용량 간단한 휴대용 크로스 바디 서류 가방"
data3_ko = "고양이의 눈 목걸이 여성 패션 2024 새로운 간단한 맞춤형 실버 체인 액세서리 여성의 짧은 쇄골 체인"

Data Preprocessing (Simpler ver.)


In [ ]:
import pandas as pd
product_title_inst = "### Instruction: Given a raw source for a product, generate a coherent and relevant product title in Korean.\n"
detailed_description_inst = "### Instruction: Given a raw source for a product, generate five detailed descriptions in Korean. Each description should be relevant to the raw source.\n"
related_keywords_inst = "### Instruction: Given a raw source for a product, generate related search keywords in Korean that are relevant to the raw source and practically usable.\n"


raw_source_format = "### Raw Source:"


new_data = [
    {'raw_source': data1_ko},
    {'raw_source': data2_ko},
    {'raw_source': data3_ko}
]
base_df = pd.DataFrame(new_data)


title_data = []
description_data = []
keywords_data = []


for index, row in base_df.iterrows():
    title_data.append({
        'text': product_title_inst + raw_source_format + row['raw_source'] + "\n### Product Title:"
    })
    description_data.append({
        'text': detailed_description_inst + raw_source_format + row['raw_source'] + "\n### Detailed Description:"
    })
    keywords_data.append({
        'text': related_keywords_inst + raw_source_format + row['raw_source'] + "\n### Related Keywords:"
    })


title_df = pd.DataFrame(title_data)
description_df = pd.DataFrame(description_data)
keywords_df = pd.DataFrame(keywords_data)


In [ ]:
title_df['type'] = 'Product Title'
description_df['type'] = 'Detailed Description'
keywords_df['type'] = 'Related Keywords'

In [ ]:
combined_df = pd.concat([title_df, description_df], ignore_index=True)

In [ ]:
combined_df = pd.concat([combined_df, keywords_df], ignore_index=True)


# **전처리 완료한 데이터**



In [ ]:
pd.set_option('display.max_colwidth', None)
combined_df

# **Llama2 Base 시연**

In [ ]:
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import time


pipe_product_title = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=256)
pipe_detailed_description = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=1024)
pipe_search_keywords = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=256)

results = []

#base_test = pd.read_csv(base_test_path)


pipe_product_title = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=256)
pipe_detailed_description = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=1024)
pipe_search_keywords = pipeline(task="text-generation", model=my_model, tokenizer=my_tokenizer, max_length=256)


timing_info = {
    "Product Title": {"total_time": 0, "count": 0},
    "Detailed Description": {"total_time": 0, "count": 0},
    "Related Keywords": {"total_time": 0, "count": 0}
}

results = []


for index, row in combined_df.iterrows():


    prompt = row['text']
    result = None
    start_time = time.time()

    if row['type'] == "Product Title":
        result = pipe_product_title(f"{prompt}")
        timing_info["Product Title"]["count"] += 1
    elif row['type'] == "Detailed Description":
        result = pipe_detailed_description(f"{prompt}")
        timing_info["Detailed Description"]["count"] += 1
    elif row['type'] == "Related Keywords":
        result = pipe_search_keywords(f"{prompt}")
        timing_info["Related Keywords"]["count"] += 1

    end_time = time.time()
    elapsed_time = end_time - start_time
    timing_info[row['type']]["total_time"] += elapsed_time

    if result:
        generated_text = result[0]['generated_text']
        ext = extract_response(generated_text)

        print(f"Prompt: {prompt}")
        print(f"Generated Text: {generated_text}")
        print("\n" + ext)
        print("-" * 80)

        results.append({
            'prompt': prompt,
            'type': row['type'],
            'generated_text': ext,
            'total': generated_text
        })


results_df = pd.DataFrame(results)


for text_type, info in timing_info.items():
    if info['count'] > 0:
        average_time = info['total_time'] / info['count']
        print(f"Average time for {text_type}: {average_time:.2f} seconds")







In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
for index, row in results_df.iterrows():
  print(row["type"]+"\n")
  print(row["generated_text"]+"\n")
  print("---------------------------------------------------------------")

In [ ]:
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer



pipe_product_title = pipeline(task="text-generation", model=my_model2, tokenizer=my_tokenizer2, max_length=256)
pipe_detailed_description = pipeline(task="text-generation", model=my_model2, tokenizer=my_tokenizer2, max_length=1024)
pipe_search_keywords = pipeline(task="text-generation", model=my_model2, tokenizer=my_tokenizer2, max_length=256)

results = []

for index, row in combined_df.iterrows():

    prompt = row['text']
    result = None

    if row['type'] == "Product Title":
        result = pipe_product_title(f"{prompt}")
    elif row['type'] == "Detailed Description":
        result = pipe_detailed_description(f"{prompt}")
    elif row['type'] == "Related Keywords":
        result = pipe_search_keywords(f"{prompt}")

    if result:
        generated_text = result[0]['generated_text']
        ext = extract_response(generated_text)

        print(f"Prompt: {prompt}")
        #print(f"Generated Text: {generated_text}")
        print("\n"+ext)
        print("-" * 80)

        results.append({
            'prompt': prompt,
            'type': row['type'],
            'generated_text': ext,
            'total': generated_text
        })



results_df = pd.DataFrame(results)

